<a href="https://colab.research.google.com/github/RaneenAlshehri/SDAIA-KAUST-AI-Bootcamp/blob/main/Deep_Learining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torchmetrics

import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torchmetrics
from torch.utils.tensorboard import SummaryWriter




In [ ]:
DATA_DIR = pathlib.Path("../input/sdaia-kaust-ai-august-school-2022")


train_df = pd.read_csv(DATA_DIR / "train.csv", dtype="uint8")
test_df  = pd.read_csv(DATA_DIR / "test.csv", dtype="uint8")


train_features = train_df.drop("labels", axis=1)
train_target   = train_df.loc[:, "labels"]

In [ ]:
class DataSetWithTransforms(Dataset):
    
    def __init__(self, features, target, feature_transforms=None):
        super().__init__()
        self._features = features
        self._target = torch.from_numpy(target).long()
        self._feature_transforms = feature_transforms
        
    def __getitem__(self, index):
        if self._feature_transforms is None:
            features = self._features[index]
            #feature = torch.rashape(feature, (32,32,3))
        else: 
            features = self._feature_transforms(self._features[index])
        target = self._target[index]
        return (features, target) 
    
    def __len__(self):
        n_samples, _ = self._features.shape
        return n_samples

In [ ]:
class DataSetTest(Dataset):
    
    def __init__(self, features, feature_transforms=None):
        super().__init__()
        self._features = features
        self._feature_transforms = feature_transforms
        
    def __getitem__(self, index):
        if self._feature_transforms is None:
            features = self._features[index]
            #feature = torch.rashape(feature, (32,32,3))
        else: 
            features = self._feature_transforms(self._features[index])
        return (features) 
    
    def __len__(self):
        n_samples, _ = self._features.shape
        return n_samples

In [ ]:
# data augmentation should only apply to training data
_feature_transforms = transforms.Compose([
    transforms.Lambda(lambda array: array.reshape((32, 32))),
    transforms.ToPILImage(),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), shear=15, scale=(1.0, 1.1)),
    transforms.ToTensor(),
])


train_dataset = DataSetWithTransforms(train_features.values, train_target.values, _feature_transforms)

In [ ]:
train_loader=DataLoader(train_dataset,4)

In [ ]:
for batch in train_loader:
    print(batch[1].shape)

In [ ]:
# conveting test data only to tensor images should only apply to training data
_feature_transforms_test = transforms.Compose([
    transforms.Lambda(lambda array: array.reshape((32, 32))),
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

test_dataset = DataSetTest(test_df.values, _feature_transforms_test)

In [ ]:
test_loader=DataLoader(test_dataset,4)

In [ ]:
for batch in test_loader:
    print(batch.shape)
    break

In [ ]:
/ 

In [ ]:
device=torch.device("cpu")
net=NeauralNetworks()
net=net.to(device)

optimizer=optim.SGD(net.parameters(),lr=0.03,momentum=0.5)

num_epochs=80 

los=[]


for i in range(num_epochs):
    total_loss=0
    for batch in train_loader:
        image=batch[0].to(device)
        label=batch[1].to(device)
    
        logits=net(image) #DOING THE RTAINING THEN GETTIN
        loss=F.cross_entropy(logits,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    
    los.append(total_loss)
    
plt.plot(los)   

      

In [ ]:
prediction_list = list()
with torch.no_grad():
    for i,batch in enumerate(test_loader):
        data=batch
        data= data.to(device)
        logits = net(data)
        out=torch.argmax(logits, dim=1)
        prediction_list.append(out)
prediction_list = torch.cat(prediction_list).cpu()

test_features = pd.read_csv(DATA_DIR / "test.csv")
_ = (pd.DataFrame
       .from_dict({"Id": test_features.index, "Category": prediction_list})
       .to_csv("submission.csv", index=False))